In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/home/ldaquan1996/Regression_ML_EndtoEnd/.venv/bin/python
3.0.4
/home/ldaquan1996/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/xgboost/__init__.py


In [3]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

/home/ldaquan1996/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("/home/ldaquan1996/Regression_ML_EndtoEnd/data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("/home/ldaquan1996/Regression_ML_EndtoEnd/data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [7]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [8]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("/home/ldaquan1996/Regression_ML_EndtoEnd/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

[I 2026-01-06 00:04:20,661] A new study created in memory with name: no-name-a6e93288-9a13-49b9-bab7-a05245638cd7
[I 2026-01-06 00:05:05,785] Trial 0 finished with value: 69430.44252617517 and parameters: {'n_estimators': 801, 'max_depth': 9, 'learning_rate': 0.12353983135004505, 'subsample': 0.6551121103069528, 'colsample_bytree': 0.5549216217520294, 'min_child_weight': 8, 'gamma': 3.973627689327381, 'reg_alpha': 0.0031570329044474273, 'reg_lambda': 4.469821609475818}. Best is trial 0 with value: 69430.44252617517.
[I 2026-01-06 00:05:19,185] Trial 1 finished with value: 72496.11187904642 and parameters: {'n_estimators': 417, 'max_depth': 6, 'learning_rate': 0.09538174886894928, 'subsample': 0.6111942777513217, 'colsample_bytree': 0.8846190574698031, 'min_child_weight': 1, 'gamma': 2.7635590373428185, 'reg_alpha': 0.003474724751867644, 'reg_lambda': 1.6073038306155039e-06}. Best is trial 0 with value: 69430.44252617517.
[I 2026-01-06 00:05:30,661] Trial 2 finished with value: 83595.49

Best params: {'n_estimators': 990, 'max_depth': 10, 'learning_rate': 0.05011673985228474, 'subsample': 0.7540920768280955, 'colsample_bytree': 0.5149026944528615, 'min_child_weight': 10, 'gamma': 3.7374531384085206, 'reg_alpha': 1.1292850289188117e-08, 'reg_lambda': 7.203762009316714}


In [9]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 30910.164630071213
RMSE: 69308.05255695073
R²: 0.9628783370426398


/home/ldaquan1996/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/xgboost/sklearn.py:1028: UserWarning: [00:15:05] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2026/01/06 00:15:12 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026/01/06 00:15:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
